# Generate structured content via Functional Calling with Nova

### 1) Setup
* Install python modules
* Import required classes/functions
* Set all static variables
* initialise clients (AWS)

In [ ]:
!pip3 install -q aws-requests-auth

from aws_requests_auth.aws_auth import AWSRequestsAuth
import boto3
import datetime
import json
import os
import requests
import urllib

API_ENDPOINT="<API_ENDPOINT from cdk>"
AWS_REGION="us-east-1"
BEDROCK_MODEL="us.amazon.nova-pro-v1:0"
BEDROCK_TOOL_MODEL="auto"
# switch to "any" if you want to force the model to "only" use the tools you provide 

aws_session_credentials = boto3.Session().get_credentials()
aws_client_bedrock = boto3.client(
    service_name="bedrock-runtime"
)

### 2) Prompt for user input

In [ ]:
print("What is your question:")
user_input = input()

### 3) Run user input against Nova
* create list of messages to send to model
* define tool configuration as JSON-schema
* call the bedrock Converse endpoint
* check the stop reason for the generation and assign the tool response to a variable

In [ ]:
llm_output_1 = aws_client_bedrock.converse(
    modelId=BEDROCK_MODEL,
    messages=[{
        "role": "user",
        "content": [{
            "text": f"""
                <user_input>
                {user_input}
                </user_input>
                <instructions>
                the current date is {datetime.datetime.today()}
                </instructions>
            """
        }]
    }],
    toolConfig={
        "tools": [
            {
                "toolSpec": {
                    "name": "weather",
                    "description": "Use this tool if the request is to get the previous, current, or future weather for a country",
                    "inputSchema": {
                        "json": {
                            "type": "object",
                            "properties": {
                                "country": {
                                    "type": "string"
                                },
                                "dates": {
                                    "type": "object",
                                    "properties": {
                                        "date_start": {
                                            "type": "string",
                                            "format": "date"
                                        },
                                        "date_end": {
                                            "type": "string",
                                            "format": "date"
                                        }
                                    },
                                    "required": [
                                        "date_start",
                                        "date_end"
                                    ]
                                }
                            },
                            "required": [
                                "country",
                                "dates"
                            ]
                        }
                    }
                }
            }
        ],
        "toolChoice": {
            f"{BEDROCK_TOOL_MODEL}": {}
        }
    }
)

tool_payload = None
if(llm_output_1['stopReason'] == "tool_use"):
    for c in llm_output_1['output']['message']['content']:
        if('toolUse' in c):
            tool_payload = c['toolUse']
    print("> tool use identified:")
    print(json.dumps(tool_payload, indent=4))
else:
    print("> no tool required to answer")

### 4) Process Answer
* check to see if a tool use was requ
* create a signed request to APIGateway
* lookup the mock response
* use the mock response in a second generation to answer the user question

In [ ]:
if(tool_payload):
    auth = AWSRequestsAuth(
        aws_access_key=aws_session_credentials.access_key,
        aws_secret_access_key=aws_session_credentials.secret_key,
        aws_token=aws_session_credentials.token,
        aws_host=urllib.parse.urlparse(API_ENDPOINT).netloc,
        aws_region=AWS_REGION,
        aws_service="execute-api"
    )

    api_response = requests.post(
        API_ENDPOINT,
        json=tool_payload['input'],
        auth=auth
    ).text

    print("> Mock API Gateway Response:")
    print(json.dumps(json.loads(api_response), indent=4))

    llm_output_2 = aws_client_bedrock.converse(
        modelId=BEDROCK_MODEL,
        messages=[{
            "role": "user",
            "content": [{
                "text": f"""
                    <api_payload>{tool_payload['input']}</api_payload>
                    <api_response>{api_response}</api_response>
                    <user_question>{user_input}</user_question>
                    <instructions>
                    - Give a short and concise answer to <user_question> using whats in <api_response>
                    - <api_payload> is the request to lookup information on an external service
                    - <api_response> is the response from the external service
                    </instructions>
                """
            }]
        }]
    )
    output_message = llm_output_2['output']['message']
    for c in output_message['content']:
        print("> Nova Response (with tool):")
        print(c['text'])
else:
    for c in llm_output_1['output']['message']['content']:
        print("> Nova Response (without tool):")
        print(c['text'])